In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from astropy.table import Table
import gravity_obs.p2_tools as p2t

In [3]:
table_path = '../../gvprep/tabs/star_k9mag_grp10-12mag_south_gaia.ipac'
tb = Table.read(table_path, format='ascii.ipac')

In [4]:
tb

name,ra_2000_hms,dec_2000_dms,ra_2000,dec_2000,H,K,parallax,pm,pmra,pmdec,radial_velocity,Grp,G,ra_2016,dec_2016,source_id
,,,,,mag,mag,mas,mas / yr,mas / yr,mas / yr,km / s,mag,mag,deg,deg,
str28,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
Gaia DR3 2312711078532837632,00:00:00.263,-34:41:52.377,0.001095554660312786,-34.6978823634163,9.232000351,9.1120004654,2.1987,23.731,18.349,-15.049,23.63,10.490212,11.143433,0.00119474281,-34.69794924793,2312711078532837632
Gaia DR3 2312704477167095040,00:00:01.944,-34:52:36.848,0.008098460835500598,-34.87690218354403,9.9670000076,9.8240003586,0.2713,12.961,0.021,-12.961,-24.27,11.498783,12.227124,0.00809857507,-34.8769597872,2312704477167095040
Gaia DR3 2313154662754924288,00:00:03.559,-32:59:58.399,0.014830507474218066,-32.999555159954866,9.5850000381,9.4890003204,0.3307,12.684,3.729,-12.123,-6.35,11.241921,11.992699,0.01485026969,-32.99960904071,2313154662754924288
Gaia DR3 2333399798559115904,00:00:05.231,-27:42:37.587,0.02179436484568733,-27.710440759851195,10.06799984,9.9809999466,0.4974,7.992,-2.346,-7.64,-8.89,11.334542,11.964486,0.02178258956,-27.71047471333,2333399798559115904
Gaia DR3 2334693820665572096,00:00:07.590,-25:59:42.244,0.03162554655028096,-25.995067807559526,9.4910001755,9.3940000534,0.6604,10.278,10.076,2.029,1.52,10.769584,11.4098,0.03167536748,-25.9950587919,2334693820665572096
Gaia DR3 2307546225740273024,00:00:07.600,-38:19:10.119,0.03166747236191719,-38.319477606077214,9.6510000229,9.5539999008,4.0991,35.651,29.839,19.508,8.77,10.300985,10.707719,0.03183650813,-38.31939090275,2307546225740273024
Gaia DR3 2332868008592476800,00:00:11.329,-29:50:42.518,0.04720593358205661,-29.84514389935682,9.8500003815,9.7349996567,4.6467,76.815,9.644,-76.207,-8.55,10.92845,11.494892,0.04725534852,-29.84548259646,2332868008592476800
Gaia DR3 2314829528201886592,00:00:12.037,-30:20:50.362,0.05015601411339164,-30.347322807001515,9.6470003128,9.5629997253,3.601,31.848,-4.128,-31.579,-6.26,10.375031,10.823464,0.0501347539,-30.34746315853,2314829528201886592


In [5]:
tb.add_index('name')

In [ ]:
from gravity_obs import ob_add_target
def single_on_for_target(grav, target, folder_name=None,
                         usename=False, verify=False):
    ao_mode = "LGS_VIS"
    tel_alpha = target['ra_2000_hms']
    tel_delta = target['dec_2000_dms']

    if usename:
            obname = f"{ao_mode}_{target['name']}".replace(' ', '_')
    else:
            obname = f"{ao_mode}_{tel_alpha.split('.')[0].replace(':', '')}{tel_delta.split('.')[0].replace(':', '')}_{target['Grp']:4.1f}_{target['K']:4.1f}".replace(' ', '_')


    ob, obV = grav.create_OB(obname, folder_name=folder_name, overwrite=True)
    obsDes = ob['obsDescription']
    obsDes['name'] = target['name']
    obsDes['userComments'] = 'Single on-axis observation test'

    ob_add_target(ob, target['name'], 
                    ra = target['ra_2000_hms'], 
                    dec = target['dec_2000_dms'],
                    pma = target['pmra'],
                    pmd = target['pmdec'])
    ob, obV = grav.save_OB(obname, folder_name=folder_name)


    dit, ndit = 3, 40
    grav.add_GRAVITY_single_onaxis_acq(name=obname, folder_name=folder_name,
                                        sc_name=target['name'],
                                        sc_kmag=target['K'],
                                        sc_hmag=target['H'],
                                        ngs_mag=target['Grp'],
                                        ao_mode=ao_mode,
                                        plx=target['parallax']/1e3,
                                        baseline=['UTs'])
    grav.add_GRAVITY_single_obs_exp(obname, folder_name=folder_name,
                                    dit=dit, ndit_obj=ndit, ndit_sky=ndit,)

    # optional verification
    if verify:
        obID = ob['obId']
        response, _ = grav.api.verifyOB(obID,True)
        if response['observable']:
            print('*** Congratulations. Your OB' , obID, ob['name'], 'is observable!')
        else:
            print('OB', obID, 'is >>not observable<<. See messages below.')
            print(' ', '\n  '.join(response['messages']))

In [4]:
grav = p2t.p2api_GRAVITY('116.29H7.001', 'GPLUSTeam', 'GPAOwFRINGES', no_warning=True)
root_folder = 'TTR-116.0046 October First Light'
grav.set_rootFolder(root_folder)

In [5]:
folder_name = 'single_onaxis'
try:
    grav.get_folder(folder_name)
except Exception:
    grav.create_folder(folder_name, overwrite=True)

In [ ]:
target_names = ["Gaia DR3 2312711078532837632", "Gaia DR3 2334558821253472256"]
folder_name = 'onaxis_ob_test'

for obj_name in target_names:
    target = tb.loc[obj_name]
    single_on_for_target(grav, target, folder_name=folder_name)